##  Calculate Yearly DPLE variables

In [1]:
import xarray as xr
import numpy as np
import metabolic as mi
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')

In [2]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """

    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

In [3]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
tres = 'monthly'
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)

In [4]:
for varr in ['O2','AOU','T','S']:
    print(varr)
    filename = f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_Eo_space_vertical_mean_1948-2017_{tres}.nc'
    ds = xr.open_dataset(filename)
    dsf = xr.Dataset()
    for cc in [0,6,12]:
        print(cc)
        for layer in ['0-200m', '200-600m']:
            dsf[f'Phi_{varr}_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{varr}_{cc}_{layer}'], 'time', ['year', 'mon'], [np.arange(70), np.arange(12)]).mean(dim='mon')
    dsf.load()
    dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_Eo_space_vertical_mean_1948-2017_yearly.nc', mode='w')
    #
    filename = f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_Eo_space_LME_1948-2017_{tres}.nc'
    ds = xr.open_dataset(filename)
    dsf = xr.Dataset()
    for cc in [0,6,12]:
        print(cc)
        for layer in ['0-200m', '200-600m']:
            dsf[f'Phi_{varr}_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{varr}_{cc}_{layer}'], 'time', ['year', 'mon'], [np.arange(70), np.arange(12)]).mean(dim='mon')
    dsf.load()
    dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_Eo_space_LME_1948-2017_yearly.nc', mode='w')

O2
0
6
12
0
6
12
AOU
0
6
12
0
6
12
T
0
6
12
0
6
12
S
0
6
12
0
6
12
